In [ ]:
# using Pkg
# Pkg.add("https://github.com/MasanoriKanamaru/Astroshaper")
# Pkg.update("Astroshaper")

In [ ]:
# Pkg.test("Astroshaper")

In [ ]:
using Revise
using Astroshaper
using Plots

using FileIO
using JLD2
using Profile

In [ ]:
shapedir = "/Users/masanorikanamaru/Documents/shape/ryugu"

shapename = "ryugu_v252_f500.obj"
# shapename = "ryugu_v752_f1500.obj"
# shapename = "ryugu_test.obj"

# shapename = "SHAPE_SPC_49k_v20190802.obj"
# shapename = "SHAPE_SPC_200k_v20190802.obj"
# shapename = "SHAPE_SPC_800k_v20190802.obj"
# shapename = "SHAPE_SPC_3M_v20190802.obj"

# shapename = "SHAPE_SFM_49k_v20180804.obj"
# shapename = "SHAPE_SPC_49k_v20200323.obj"
# shapename = "SHAPE_SPC_3M_v20200323.obj"

shapepath = joinpath(shapedir, shapename)
@show shapepath;

In [ ]:
@time shape = setShapeModel(shapepath; scale=1000, find_visible_faces=true, save_shape=false)

# shape = setShapeModel(splitext(shapepath)[1] * ".jld2")

In [ ]:
# showshape(shape)

In [ ]:
params_orbit = Dict()

params_orbit[:a] = 1.18956373  # semi-mojor axis [AU]
params_orbit[:e] = 0.19027921  # eccentricity
params_orbit[:I] = 5.8840222   # inclination [deg]
params_orbit[:Ω] = 251.589203  # longitude of the ascending node [deg]
params_orbit[:ω] = 211.435963  # argument of periapsis [deg]
params_orbit[:Φ] = 21.9353799  # mean anomaly [deg]

params_orbit[:μ] = GM☉ + 30.0

orbit = OrbitalElements(params_orbit)

In [ ]:
params_spin = Dict()
params_spin[:α] = 96.4
params_spin[:δ] = -66.4
params_spin[:T] = 7.63262

spin = setSpinParams(params_spin, orbit)

In [ ]:
dt = spin.T / 72
times = Vector(0:dt:orbit.T);

In [ ]:
# @time τ̄ = getNetTorque(shape, orbit, spin, times)
@time τ̄ = getNetTorque_shadowing(shape, orbit, spin, times)

C = 4.039541372643629e16
ω̇, ωε̇, ωψ̇ = torque2rate(τ̄, spin, C)  # [deg/day/day]

@show shapename
@show τ̄
@show ω̇, ωε̇, ωψ̇
@show getTimeScale(3.5, 7.63262, ω̇);  # 3.5時間から7.6時間まで減速する時間スケール [Myr]

# 熱物理モデル

In [ ]:
shape = setShapeModel(splitext(shapepath)[1] * ".jld2")

In [ ]:
params_thermo = ParamsThermo(
    A_B = 0.04,
    A_TH = 0.,
    k = 0.1,
    ρ = 1270.,
    Cₚ = 600.,
    ϵ = 1.,
    P = spin.T,
    Δt = spin.T/72,
    t_bgn = 0.,
    t_end = spin.T*10,
    # t_end = orbit.T,
    Δz = 0.02,
    z_max = 0.6,
)

In [ ]:
@time τ̄ = run_YORP(shape, orbit, spin, params_thermo)

## ryugu_v252_f500.obj
##      0.192743 seconds (1.00 k allocations: 289.609 KiB) 10 自転分
##     24.054642 seconds (1.00 k allocations: 289.609 KiB)　1 公転分

In [ ]:
C = 4.039541372643629e16
ω̇, ωε̇, ωψ̇ = torque2rate(τ̄, spin, C)  # [deg/day/day]

@show shapename
@show τ̄
@show ω̇, ωε̇, ωψ̇
@show getTimeScale(3.5, 7.63262, ω̇);  # 3.5時間から7.6時間まで減速する時間スケール [Myr]

In [ ]:
# @profile τ̄ = run_YORP(shape, orbit, spin, params_thermo)
# Profile.print()

In [ ]:
# temps = get_surface_temperature(shape);

In [ ]:
# open("ryugu_test_temps.txt","w") do f
#     for smesh in shape.smeshes
#         println(f, smesh.Tz[begin], ", ", smesh.flux.sun, ", ", smesh.flux.scat, ", ", smesh.flux.rad)
#     end
# end

In [ ]:
# shapename = "ryugu_v252_f500.obj"

# Thermophysical parameters
# -------------------------
# A_B   : 0.04
# A_TH  : 0.0
# k     : 0.1
# ρ     : 1270.0
# Cₚ    : 600.0
# ϵ     : 1.0
# P     : 27477.432
# l     : 0.21287051812296282
# Γ     : 276.04347483684523
# Δt    : 0.01388888888888889
# t_bgn : 0.0
# t_end : 1490.1074827395094
# Nt    : 107288
# Δz    : 0.09395382778392625
# z_max : 2.8186148335177874
# Nz    : 31
# λ     : 0.12520702099737538

# 影 + 熱伝導
# τ̄ = [2.166292988343463, -1.8232052646633534, 0.12149914462601369]
# (ω̇, ωε̇, ωψ̇) = (-2.3559241586480615e-6, 7.198547956478111e-6, 2.903520943282301e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5543304681988876

# 影 + 熱伝導 + 放射の再吸収
# τ̄ = [2.1867628755835007, -1.9130691894522611, 0.20980627031427984]
# (ω̇, ωε̇, ωψ̇) = (-3.3918521233894786e-6, 7.818508184547549e-6, 2.964269717748932e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0796121314667426

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱
# τ̄ = [2.1865019814933127, -1.9132227674055484, 0.20659523329863488]
# (ω̇, ωε̇, ωψ̇) = (-3.358599466983103e-6, 7.826075560400724e-6, 2.9640911737278277e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0903011021560833

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1769952611580217, -1.9075575309460082, 0.11690136500927037]
# (ω̇, ωε̇, ωψ̇) = (-2.4174445056821524e-6, 7.953525830274134e-6, 2.9524204264352846e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5147750825077182

In [ ]:
# shapename = "ryugu_v752_f1500.obj"

# 影
# τ̄ = [3.3097974041391374, -1.7111464770712586, 0.12728322411992796]
# (ω̇, ωε̇, ωψ̇) = (-1.5003553780529698e-6, 9.426867138482173e-7, 3.9434066533743185e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.440678224718057

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.165370019439715, -1.8680106417353224, 0.027423863292429056]
# (ω̇, ωε̇, ωψ̇) = (-1.433047521048842e-6, 7.77033687219021e-6, 2.923188963996669e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.5553128188466436

In [ ]:
# shapename = "ryugu_test.obj"

# 影
# τ̄ = [3.2623680861875415, -1.7632177354631766, 0.2112963317251145]
# (ω̇, ωε̇, ωψ̇) = (-2.484150619384357e-6, 1.5205017761112165e-6, 3.9220246601817255e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.4740993045985396

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1336218786667724, -1.878807112180092, 0.04045730505339008]
# (ω̇, ωε̇, ωψ̇) = (-1.6056650245345357e-6, 7.99631943219241e-6, 2.8978423396048164e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.280603142373392

In [ ]:
surf_temp = [T[begin] for T in Ts];

In [ ]:
plot(framestyle=:box, legend=false, size=(600,400))
plot!(xlims=(0, t_max/P))
plot!(ylims=(0, 400))

ts = collect(0:Δτ:t_max/P)
zs = collect(0:Δz:x_max/l)

plot!(ts, surf_temp)

xlabel!("Time / Rotation period")
ylabel!("Surface temperature [K]")

# View factorの計算時間

In [ ]:
plot(framestyle=:box, legend=false)


Ns = [500,      1500,     5932,     49152,      196608]    # 786432,     3145728
ts = [0.002659, 0.027669, 0.473278, 164.478600, 10829.774176]    # 3600*24*10, 3600*24*1000
plot!(Ns, ts, marker=(:circle))

plot!(xticks=[10^3, 10^4, 10^5])
plot!(xlabel="Number of facets")
plot!(ylabel="Calculation time [sec]")
plot!(xaxis=:log, yaxis=:log)

# TPM出力からYORPトルクを計算する

## vtkファイルの読込み

最終的には自分で書いた方がよさそう...

VTK Python tutorial --> https://kitware.github.io/vtk-examples/site/Python/

In [ ]:
using Plots
using FileIO
using LinearAlgebra
using DataFrames
using Astroshaper

In [ ]:
isvtk(filename) = splitext(filename)[end] == ".vtk"
filename2phase(filename) = parse(Int64, splitext(filename)[begin][79:end]);

In [ ]:
vtkfiles = filter(f -> isvtk(f), readdir("."))
deleteat!(vtkfiles, 361);

In [ ]:
@show vtkname = vtkfiles[1]
@show filename2phase(vtkname);

In [ ]:
using PyCall

@show PyCall.pyversion
@show PyCall.pyprogramname
@show PyCall.libpython;

In [ ]:
# pyimport_conda("vtk", "vtk")
vtk = pyimport("vtk");

In [ ]:
function get_temps(vtkfile)
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(vtkfile)
    reader.ReadAllScalarsOn()
    reader.ReadAllVectorsOn()
    reader.ReadAllTensorsOn()
    reader.Update()
    vtkdata = reader.GetOutput()
    
    # @show vtkdata.GetNumberOfPoints()
    # @show vtkdata.GetNumberOfCells()
    # @show vtkdata.GetPoint(0)
    
    celldata = vtkdata.GetCellData()
    temps = celldata.GetScalars()
    nodes = vtkdata.GetPoints().GetData()
    
    temps
end

## 温度分布から熱トルクを求める

In [ ]:
get_df(ϵ, T, n̂, dS) = - 2/3 * ϵ * σ_SB * T^4 / c₀ * dS .* n̂
get_df(ϵ, T, mesh) = get_df(ϵ, T, mesh.normal, mesh.area)

get_dτ(ϵ, T, mesh) = mesh.center × get_df(ϵ, T, mesh)

In [ ]:
shapepath = "/Users/masanorikanamaru/Documents/shape/ryugu/SHAPE_SFM_49k_v20180804.obj"
shape = setShapeModel(shapepath; scale=1000, find_visible_faces=true);

In [ ]:
temps = get_temps(vtkfiles[1])
dfs = get_df.(1, temps, shape.smeshes)
dτs = get_dτ.(1, temps, shape.smeshes)
τ = sum(dτs)
@show τ;

## １自転分のスナップショットに対して計算

In [ ]:
ϕs = []
τs = []
for vtkfile in vtkfiles
    ϕ = filename2phase(vtkfile)
    temps = get_temps(vtkfile)
    
    dτs = get_dτ.(1, temps, shape.smeshes)
    τ = sum(dτs)
    
    push!(ϕs, ϕ)
    push!(τs, τ)
end

In [ ]:
params_orbit = Dict()

params_orbit[:a] = 1.18956373  # semi-mojor axis [AU]
params_orbit[:e] = 0.19027921  # eccentricity
params_orbit[:I] = 5.8840222   # inclination [deg]
params_orbit[:Ω] = 251.589203  # longitude of the ascending node [deg]
params_orbit[:ω] = 211.435963  # argument of periapsis [deg]
params_orbit[:Φ] = 21.9353799  # mean anomaly [deg]

params_orbit[:μ] = GM☉ + 30.0

orbit = OrbitalElements(params_orbit);

In [ ]:
params_spin = Dict()
params_spin[:α] = 96.4
params_spin[:δ] = -66.4
params_spin[:T] = 7.63262

spin = setSpinParams(params_spin, orbit)

In [ ]:
## Body-fixed frame -> orbital plane frame
# τs = Astroshaper.body_to_orbit.(τs, spin.γ, spin.ε, ϕs);

In [ ]:
function get_df(ϕs, τs, spin)
    τω = [Astroshaper.getτω(τ, spin) for τ in τs]
    τε = [Astroshaper.getτε(τ, spin) for τ in τs]
    τψ = [Astroshaper.getτψ(τ, spin) for τ in τs]
    
    df = DataFrame(ϕ=ϕs, τω=τω, τε=τε, τψ=τψ)
    # sort!(df, [:ϕ])
end

In [ ]:
df = get_df(ϕs, τs, spin);

In [ ]:
marker = (:circle, 2, 0.9, :orange, stroke(0, 0., :black, :dot))

p1 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(df.ϕ, df.τω, marker=marker)

p2 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(df.ϕ, df.τε, marker=marker)

p3 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(df.ϕ, df.τψ, marker=marker)

plot!(xlabel="Phase [deg]")
# plot!(ylabel="τ / C [10^-6 deg/day/day]")

layout = @layout [
    a
    b
    c
]

# title = ["Type Ⅱ cases" "Type Ⅳ cases"]
plot(p1, p2, p3, layout=layout)

plot!(framestyle=:box, legend=false)
plot!(size=(800,600))
plot!(xtickfontsize=10, ytickfontsize=10)
# plot!(xlims=(0,spin.T/3600), xticks=(0:1:spin.T/3600))
# plot!(ylims=(-20,20), yticks=(-20:5:20))

In [ ]:
# for (ϕ, τ) in zip(ϕs, τs)
#     println(ϕ, " : ", τ)
# end

In [ ]:
τs

In [ ]:
sum(τs)/360

In [ ]:
myτs

In [ ]:
sum(myτs)/360

In [ ]:
mydf = get_df(0:360, myτs, spin);

In [ ]:
marker = (:circle, 2, 0.9, :orange, stroke(0, 0., :black, :dot))

p1 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(mydf.ϕ, mydf.τω, marker=marker)

p2 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(mydf.ϕ, mydf.τε, marker=marker)

p3 = hline([0], line=(1, :solid, :grey), label=false)
scatter!(mydf.ϕ, mydf.τψ, marker=marker)

plot!(xlabel="Phase [deg]")
# plot!(ylabel="τ / C [10^-6 deg/day/day]")

layout = @layout [
    a
    b
    c
]

# title = ["Type Ⅱ cases" "Type Ⅳ cases"]
plot(p1, p2, p3, layout=layout)

plot!(framestyle=:box, legend=false)
plot!(size=(800,600))
plot!(xtickfontsize=10, ytickfontsize=10)
# plot!(xlims=(0,spin.T/3600), xticks=(0:1:spin.T/3600))
# plot!(ylims=(-20,20), yticks=(-20:5:20))

In [ ]:
time_start = - 3600 * 24
dt = spin.T / 360
times = Vector(time_start:dt:time_start+spin.T)

# myτs = Astroshaper.getTorqueVsTime(shape, orbit, spin, times)
myτs = Astroshaper.getTorqueVsTime_shadowing(shape, orbit, spin, times)

# @time τ̄ = getNetTorque(shape, orbit, spin, times)
# @time τ̄ = getNetTorque_shadowing(shape, orbit, spin, times)

# C = 4.039541372643629e16
# ω̇, ωε̇, ωψ̇ = torque2rate(τ̄, spin, C)  # [deg/day/day]

# @show τ̄
# @show ω̇, ωε̇, ωψ̇
# @show getTimeScale(3.5, 7.63262, ω̇);  # 3.5時間から7.6時間まで減速する時間スケール [Myr]

#   0.061454 seconds (1 allocation: 32 bytes)
# τ̄ = [4.939408157483719, -7.864118728020829, -0.6208459855848031]
# (ω̇, ωε̇, ωψ̇) = (-1.0989356075969878e-6, 5.279549296388269e-5, 8.320493047921385e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 3.3322104363874914

In [ ]:
u = Astroshaper.solveKeplerEquation2(orbit, -3600*24)
r = Astroshaper.get_r(orbit, u) / AU
Astroshaper.orbit_to_inertia(r, orbit)

In [ ]:
C = 4.039541372643629e16

In [ ]:
P = 7.632613738534321 * 3600
albedo = 0.0146  # geometric albedo

λ = 179.3
β = -87.4

yyyy = 2018
mm = 6
dd = 30
localk = [-0.690068325, 0.699534161, -0.090245988];  # 座標はリュウグウ位置 (X,Y,Z) [AU]

# 滝田コードでは、2018年6月30日（ユリウス通日で数えている）
# 金丸コードでは、2018年7月1日
# 実際には１２時間くらいの時間差になっている

In [ ]:
const double aa = 1.19;									// 軌道長半径
const double e = 0.19;									// 離心率
const double LAN = 251.6;								// longitude of acending node
const double I = 5.88;									// 黄道傾斜角 inclination
const double PERI = 211.4;								// 近点引数 argument of perihelion
const double p_t = 2456376.135073920119*24.0 * 3600;	// 近点通過時刻（ここではユリウス秒で設定）
const double Mast = 7.63e+11;							// 小惑星の質量 ここは仮定だが、影響はない。

In [ ]:
# using FileIO, MeshIO
# mesh = load(shapepath)

In [ ]:
# using VTKView

In [ ]:
# using VTKDataTypes
# using VTKDataIO

In [ ]:
using VTKDataTypes
using WriteVTK
using PyCall
using LightXML
# using Iterators

@pyimport vtk.util.numpy_support as vtkns
@pyimport vtk as vtk
@pyimport numpy as np

In [ ]:
# include("/Users/masanorikanamaru/Documents/GitHub/VTKDataIO.jl/src/vtkreaders.jl")

In [ ]:
# read_vtk(vtkname)

In [ ]:
# read_static_vtk